## Extraction of data from JSON file and Data Exploration

In [1]:
# Import all the necessary libraries

import json 
import csv
import pandas as pd
import numpy as np

In [2]:
# Opening maps_data and search_data JSON file and loading the data 
# into the variable serach and maps 

search = []
maps =   []

for line in open('D://Jupiter/training/maps_data.json', 'r'):
    maps.append(json.loads(line))
    
for line in open('D://Jupiter/training/search_data.json', 'r'):
    search.append(json.loads(line))


In [3]:
# Read merchant_labels into a DataFrame

merch_labels = pd.read_csv('D://Jupiter/training/merchant_labels.csv')

In [6]:
# Extracting merchant name and type from maps_data

def extract_maps(maps):
    # Initialize merchant name and type to read value from the lists

    merch_type = []
    merch_name = []
    reviews    = []
    
    for merch in maps:
        rev = []
        try:
            merch_name.append(merch['merchant_name'])
        except:
            merch_name.append('None')

        try:
            #for i in merch['place_results']['type']:           
            merch_type.append(merch['place_results']['type'])
        except:
            merch_type.append('')
        
        try:
            for i in merch['place_results']['user_reviews']['most_relevant']:
                rev.append(i['description'])
        except:
            rev= ''
        reviews.append(rev)
    return merch_name, merch_type, reviews 
    

In [7]:
def extract_search(search):
    
    # Initialize snippets to read value from the lists
    
    snippets   = []
    title      = []
    #merch_name = []
    
    for merch in search:
        snip = []
        t = []
        try:
            for i in range(0,4): 
                snip.append(merch['organic_results'][i]['snippet'])
                t.append(merch['organic_results'][i]['title'])
        except:
            snip = ''
            t = ''
        #try:
        #    merch_name.append(merch['merchant_name'])
        #except:
        #    merch_name.append('None')
        snippets.append(snip)
        title.append(t)
    return snippets, title

In [8]:
merch_name_maps, merch_type, reviews = extract_maps(maps)
snippets,titles = extract_search(search)

In [9]:
# Storing the values of merchant name and type to a DataFrame

data = {'merchant_name': merch_name_maps, 'merchant_type': merch_type, 'reviews': reviews, 'snippets':snippets,
       'title':titles}
#data_search = {'merchant_name': merch_name_search}
merchant_df = pd.DataFrame(data)

In [11]:
# Join the Dataframe with features extracted from Google search with Merchant Labels

df_merged = pd.merge(merch_labels, merchant_df, on=['merchant_name'],
              how='inner')

In [12]:
df_merged

,merchant_name,category,merchant_type,reviews,snippets,title
0,PONNU SUPER MARKET,Other,[Supermarket],[A popular place for all day to day items. The...,"[Ponnu Super Market, Peravallur - No 155a, Pap...","[Ponnu Super Market, Peravallur - Home | Faceb..."
1,Infinite 4,Entertainment,,,[INFINITE 4. tournament. Home · Attendees234 ·...,"[INFINITE 4 | Events - Smash.gg, Limits to Inf..."
2,CHENNAI METRO RAIL LTD,Travel,"[Corporate office, Metropolitan train company]",[This office and also the service of metro tra...,"[Hence, it is brought to the notice of the can...","[CMRL - WELCOME TO CHENNAI METRO RAIL, Chennai..."
3,SHEETAL RESTAURANT,Food & Grocery,"[South Indian restaurant, North Indian restaur...","[A regular Udupi place, good for quick food, q...",[Sheetal Restaurant Mumbai; Sheetal Restaurant...,"[Sheetal Restaurant, Khar, Mumbai - Bandra Wes..."
4,SHRI SADGURU SERVICE S,Housing,[Services],[Very rude people!!! Do not give your bike her...,[Shri Sadguru Service Station in Shivaji Nagar...,"[Shri Sadguru Service Station, Shivaji Nagar -..."
5,Ornat Foods Private Limited,Food & Grocery,,,[Ornat Foods Private Limited is a Private inco...,"[ORNAT FOODS PRIVATE LIMITED - Company, direct..."
6,JEYACHANDRAN TEXTILES,Ecommerce & Shopping,[Clothing store],[It is good place to make purchases everything...,"[Best online textile shopping in Chennai, buy ...",[Jeyachandran Textiles is one of the Best Text...
7,SHREE GOKULESH,Food & Grocery,[Gas station],[First prity good thing is service available h...,"[Shree Gokulesh Vadodara; Shree Gokulesh, Niza...","[Shree Gokulesh, Nizampura, Vadodara - Zomato,..."
8,Marine Insight,Organization,,,[Marine Insight focuses on providing informati...,"[Marine Insight - The Maritime Industry Guide,..."
9,SAI WINES,Food & Grocery,[Wine store],[All sorts of FML and IMFL available here. Sta...,"[Sai Wines in Borivali West, Mumbai is one of ...","[Sai Wines, Borivali West - Wine Retailers in ..."


In [13]:
# Function to extract features from the list and store as string 

def de_list(x):
    words = ''
    if isinstance(x,list):
        for i in x:
            words = words+ ' '+i
    else:
        words = words + x
    return words

In [14]:
# De-listing the features

df_merged['merchant_type'] = df_merged['merchant_type'].apply(lambda x: de_list(x))
df_merged['reviews'] = df_merged['reviews'].apply(lambda x: de_list(x))
df_merged['snippets'] = df_merged['snippets'].apply(lambda x: de_list(x))
df_merged['title'] = df_merged['title'].apply(lambda x: de_list(x))

In [15]:
def combining_text_wo_reviews(m_name,m_type, reviews,snippet):
    return m_name + ' ' + m_type + ' ' + snippet

def combining_text_w_reviews(m_name,m_type, reviews, snippet):
    return m_name + ' ' + m_type + ' ' + reviews + ' '+ snippet

def combining_text_title(m_name,m_type, reviews, snippet, title):
    return m_name + ' ' + m_type  + ' '+ title

def combining_snip_title(m_name,m_type, reviews, snippet, title):
    return m_name + ' ' + m_type  + ' '+ title + ' '+ snippet

def combining_snip_title_rev(m_name,m_type, reviews, snippet, title):
    return m_name + ' ' + m_type  + ' '+ title + ' '+ snippet + ' ' + reviews

In [16]:
df_merged['snip_title'] = df_merged.apply(lambda x: combining_snip_title(x.merchant_name,x.merchant_type,x.reviews, x.snippets, x.title), axis = 1)

In [17]:
df_merged.to_csv(r'D://Jupiter/training/data_set.csv', index = False)